## Code for Creating SUMMA Forcing Files
* Taken from, at some point, Bart's code for creating SUMMA forcing files / ncdfs into the correct format 

In [1]:
# import packages 
# %matplotlib widget
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
"""
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff
"""
# data packages 
# pandas is the most popular Python data analysis/manipulation package, which is built upon NumPy.
import pandas as pd
import numpy as np
import xarray as xr
# import datetime class
# an aware object can locate itself relative to other aware objects. An aware object represents a specific moment in time that is not open to interpretation.
# A naive object does not contain enough information to unambiguously locate itself relative to other date/time objects.
from datetime import datetime

import csv 
import copy 
import os.path 

## About DataFrames
A Spark DataFrame is an immutable set of objects organized into columns and distributed across nodes in a cluster. DataFrames are a SparkSQL data abstraction and are similar to relational database tables or Python Pandas DataFrames. The first step is to import csv into Pandas DataFrame.

## About Pandas

pandas has two main data structures: Series and DataFrame.

Series: a 1-dimensional labeled array that can hold any data type such as integers, strings, floating points, or Python objects. It has row/axis labels as the index.

DataFrame: a 2-dimensional labeled data structure with columns of potentially different types.
It also contains row labels as the index.


In [2]:
# Order doesnt matter
# Load csv into Python as pandas DataFrame. Anything that is a number is int type and anything
# thats NAN is object type
ds =  pd.read_csv("/Users/ianwhidden/pysumma/NLDAS2_WY22.csv")

# below is the metadata from the NLDAS website
# 61:APCP:Precipitation hourly total [kg/m^2] Hourly backward accumulated
# 205:DLWRF:Longwave radiation flux downwards (surface) [W/m^2]
# 204:DSWRF:Shortwave radiation flux downwards (surface) [W/m^2]
# 1:PRES:Surface pressure [Pa]
# 51:SPFH:2-m above ground Specific humidity [kg/kg]
# 11:TMP:2-m above ground Temperature [K]
# 33:UGRD:10-m above ground Zonal wind speed [m/s]
#zonal wind is positive if it blows from the west and negative if from the east.
# 34:VGRD:10-m above ground Meridional wind speed [m/s]
#meridional wind is positive if from the south, and negative if from the north


# remove extra columns in NLDAS data
del ds["id"]
del ds["latitude"]
del ds["longitude"]
del ds[".geo"]

ds.head()

# Print row  values of second and third using a list of indexes:
# print(ds.loc[[2,3,4]])

,system:index,longwave_radiation,pressure,shortwave_radiation,specific_humidity,temperature,time_convert,time,total_precipitation,wind_u,wind_v,windspeed
0,0,362.76,92359.62,114.504,0.008863,17.930039,10/1/21 0:00,1.630000e+12,0.0,1.73,-3.00,3.463077
1,1,362.81,92384.62,51.908,0.009004,16.750039,10/1/21 1:00,1.630000e+12,0.0,1.16,-2.75,2.984644
2,2,362.86,92409.52,0.000,0.009145,15.560000,10/1/21 2:00,1.630000e+12,0.0,0.58,-2.51,2.576141
3,3,356.85,92434.32,0.000,0.009287,14.380000,10/1/21 3:00,1.630000e+12,0.0,0.00,-2.26,2.260000
4,4,356.87,92456.54,0.000,0.009220,13.910000,10/1/21 4:00,1.630000e+12,0.0,-0.04,-2.22,2.220360


In [3]:
# Pandas DatetimeIndex.date attribute outputs an
# index object containing the date values present in each of the entries of the DatetimeIndex.
# Im fairly certain this just changes the index to the column of the datetime object.
# Format should be 'Year-Month-Day HR:MIN:SEC'
ds.index = pd.DatetimeIndex(ds['time_convert'], name='time')

ds.head()

# The timestamp is Unix timestamp and has 13 numbers

,system:index,longwave_radiation,pressure,shortwave_radiation,specific_humidity,temperature,time_convert,time,total_precipitation,wind_u,wind_v,windspeed
time,,,,,,,,,,,,
2021-10-01 00:00:00,0,362.76,92359.62,114.504,0.008863,17.930039,10/1/21 0:00,1.630000e+12,0.0,1.73,-3.00,3.463077
2021-10-01 01:00:00,1,362.81,92384.62,51.908,0.009004,16.750039,10/1/21 1:00,1.630000e+12,0.0,1.16,-2.75,2.984644
2021-10-01 02:00:00,2,362.86,92409.52,0.000,0.009145,15.560000,10/1/21 2:00,1.630000e+12,0.0,0.58,-2.51,2.576141
2021-10-01 03:00:00,3,356.85,92434.32,0.000,0.009287,14.380000,10/1/21 3:00,1.630000e+12,0.0,0.00,-2.26,2.260000
2021-10-01 04:00:00,4,356.87,92456.54,0.000,0.009220,13.910000,10/1/21 4:00,1.630000e+12,0.0,-0.04,-2.22,2.220360


In [4]:
# set variable to columns in dataframe
shortwave = ds.shortwave_radiation
longwave = ds.longwave_radiation
precip = ds.total_precipitation
pressure = ds.pressure
airtemp = ds.temperature
windspd = ds.windspeed
# spechum = niwot_spechum
spechum = ds.specific_humidity
# The actual time and date gets converted into the milliseconds, and it follows the Unix time January 1st, 1970.
# Because it is the date when the time for the Unix computer started.
# The NLDAS data starts at 00:00 GMT on Friday, October 1, 2021 and
# ends at Thu Sep 29 2022 23:00:00 GMT+0000
timestamp = ds.index

In [5]:
# USING BARTS OLD CODE FOR CREATING 
# Attributes in the netCDF file, with their units and the full length name of the variable 
attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}}

# Latitude, longitude, and elevation (m) for UPLMET in the Andrews
lats = [44.2072180256268] 
lons = [-122.119450090239]
elev = 1050

bounds = ds.index

shape = (len(bounds), 1, )
dims = ('time', 'hru', )
coords = {'time': bounds}

# Time stepping
# Forcing timestep units: The user can specify the time units as <units> since <reference time>,
# where <units> is one of seconds, minutes, hours, or days and <reference time> is specified as YYYY-MM-DD hh:mm.
met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 2021-10-01'

# The variables in SUMMA that are in the forcings
summa_vars = ['airpres', 'airtemp', 'spechum', 
              'windspd', 'SWRadAtm', 'LWRadAtm', 'pptrate']

# From the docs: DataArray provides a wrapper around numpy ndarrays that uses labeled dimensions and coordinates to
# support metadata aware operations. The API is similar to that for the pandas Series or DataFrame, but DataArray objects
# can have any number of dimensions, and their contents have fixed data types.
for varname in summa_vars:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname, attrs=attrs[varname])

In [20]:
# Set set met_data with variables and hru ID

met_data['airpres' ].loc[{'hru': 0}] = pressure      #air pressure [Pa]
met_data['airtemp' ].loc[{'hru': 0}] = airtemp       #temperature [K]
met_data['windspd' ].loc[{'hru': 0}] = windspd       #wind speed [m s-1]
met_data['SWRadAtm'].loc[{'hru': 0}] = shortwave     #shortwave [W m-2]
met_data['LWRadAtm'].loc[{'hru': 0}] = longwave      #longwave [w m-2]
met_data['pptrate' ].loc[{'hru': 0}] = precip        #precip [kg m-2 s-1]
met_data['spechum' ].loc[{'hru': 0}] = spechum       #specific humidity [g/g-1]


# below is a few extra lines Cassie commented out. They may be useful later on.
# Cassie "used the attributes from the local_attributes netCDF instead of recreating them for the forcings":
#ds_local_attrs = xr.open_dataset('../data/local_attributes.nc')
#ds_template = xr.open_dataset('./CUES/cues_2016_summa_setup/forcings/cues2016.nc')

# For the model run at the Andrews I believe that I need to create the local_attributes netCDF for the forcings.
# In the below code I use the Local Attributes from the reynolds experiment.

# Open data set in read only for local attributes.
# The below is "Using the attributes from the local_attributes netCDF instead of recreating them 
# for the forcings", similar to Cassie's method. So, using hruID, lat, long, and data_step from previous NetCDF. In this case,
# 'snow_zLocalAttributes.nc'. I do not have a local attribute NetCDF file set up for the Andrews.


# NetCDF groups are not supported as part of the Dataset data model. Instead, groups can be loaded individually as Dataset
# objects. To do so, pass a group keyword argument to the open_dataset() function. 
# The group can be specified as a path-like string, e.g., to access subgroup 'bar' within group 'foo' pass '/foo/bar' as the group argument.
# In a similar way, the group keyword argument can be given to the Dataset.to_netcdf() method to write to a group in a netCDF file. When writing multiple groups in one file, pass mode='a' to Dataset.to_netcdf() to ensure that each call does not delete the file.

lsosmet = xr.open_dataset('/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/settings/snow_zLocalAttributes.nc')
#lsosmet = xr.open_dataset('/Users/ianwhidden/local_attributes.nc')
ds_local_attrs = lsosmet
met_data['hruId'] = ds_local_attrs['hruId']
# Lat and long fields don't need to be included in forcing data
met_data['latitude'] = ds_local_attrs['latitude']
met_data['longitude'] = ds_local_attrs['longitude']
# The data_step variable is the length of a timestep of the forcing data in seconds.
# 8600 seconds in a hr. This variable should be a data variable, rather than a coordinate.
#met_data['data_step'] = [3600.]
met_data['data_step'] = xr.Variable([], 3600.0)

# Write dataset contents to a netCDF file
met_data.to_netcdf('/Users/ianwhidden/pysumma/nldas2022_netCDF.nc')

# Xarray is based on the netCDF data model, so netCDF files on disk directly
# correspond to Dataset objects (more accurately, a group in a netCDF file directly
# corresponds to a Dataset object. See Groups for more.)

met_data

<xarray.Dataset>
Dimensions:       (time: 8736, hru: 1)
Coordinates:
  * time          (time) datetime64[ns] 2021-10-01 ... 2022-09-29T23:00:00
Dimensions without coordinates: hru
Data variables:
    airpres       (time, hru) float64 9.236e+04 9.238e+04 ... 9.241e+04 9.24e+04
    airtemp       (time, hru) float64 17.93 16.75 15.56 ... 16.13 16.06 15.98
    spechum       (time, hru) float64 0.008863 0.009004 ... 0.008456 0.008265
    windspd       (time, hru) float64 3.463 2.985 2.576 ... 2.349 1.91 1.863
    SWRadAtm      (time, hru) float64 114.5 51.91 0.0 0.0 ... 491.8 429.3 340.2
    LWRadAtm      (time, hru) float64 362.8 362.8 362.9 ... 339.8 339.8 339.8
    pptrate       (time, hru) float64 0.0 0.0 0.0 0.0 ... 0.0336 0.0974 0.0334
    hruId         (hru) int32 ...
    latitude      (hru) float64 ...
    longitude     (hru) float64 ...
    data_step     float64 3.6e+03
    time_convert  float64 3.6e+03

In [11]:
# view contents of snow_zLocalAttributes.nc
lsosmet
# same as met_data

<xarray.Dataset>
Dimensions:         (hru: 1, gru: 1)
Dimensions without coordinates: hru, gru
Data variables: (12/14)
    hruId           (hru) int32 ...
    gruId           (gru) int32 ...
    hru2gruId       (hru) int32 ...
    downHRUindex    (hru) int32 ...
    longitude       (hru) float64 ...
    latitude        (hru) float64 ...
    ...              ...
    tan_slope       (hru) float64 ...
    contourLength   (hru) float64 ...
    slopeTypeIndex  (hru) int32 ...
    soilTypeIndex   (hru) int32 ...
    vegTypeIndex    (hru) int32 ...
    mHeight         (hru) float64 ...
Attributes:
    history:  Converted to NetCDF using "localAttributes_ascii_to_nc.py --nc_...

In [12]:
# Code below to view contents of NetCDF
import netCDF4 as nc
fn = '/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/settings/snow_zLocalAttributes.nc'
ds = nc.Dataset(fn)
print(ds)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history: Converted to NetCDF using "localAttributes_ascii_to_nc.py --nc_template summa_zLocalAttributes_template.nc --nc_out ./out.nc --ascii_in ./reynolds/settings/snow_zLocalAttributes_shelteredSite.txt" on 2018-09-28 17:11:23

    dimensions(sizes): hru(1), gru(1)
    variables(dimensions): int32 hruId(hru), int32 gruId(gru), int32 hru2gruId(hru), int32 downHRUindex(hru), float64 longitude(hru), float64 latitude(hru), float64 elevation(hru), float64 HRUarea(hru), float64 tan_slope(hru), float64 contourLength(hru), int32 slopeTypeIndex(hru), int32 soilTypeIndex(hru), int32 vegTypeIndex(hru), float64 mHeight(hru)
    groups: 
